In [ ]:
DEBUG = False
KAGGLE = False
INFER = True

In [ ]:
import warnings
if not DEBUG:
    warnings.filterwarnings('ignore')
import os
import gc
import glob
import json
import time
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib
import optuna
import lightgbm as lgb
from joblib import Parallel, delayed
from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from numpy.random import seed
import tensorflow as tf
tf.random.set_seed(42)
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.layers import *
import kerastuner as kt
from tqdm.auto import tqdm
print('tensorflow version:', tf.__version__)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    for gpu_device in gpu_devices:
        print('device available:', gpu_device)
pd.set_option('display.max_columns', None)

In [ ]:
VER = 'v4'
CONFIG = {
    'version': VER,
    'folds': 5,
    'n_clusters': 7,
    'epochs': 10 if DEBUG else 1000,
    'patience': 4 if DEBUG else 40,
    'n_jobs': -1 if KAGGLE else 16,
    'decay': False,
    'batch_size': 1024,
    'n_quantiles': 2000,
    'seed': 2021,
    'lr': .007,
    'max_trials': 3 if DEBUG else 50,
    'lgb_max_trials': 3 if DEBUG else 50,
    'lgb_iters': 100 if DEBUG else 1000,
    'lgb_patience': 4 if DEBUG else 40,
    'comments': ''
}
DATA_PATH = '../input/optiver-realized-volatility-prediction' if KAGGLE else './data'
MDLS_PATH = f'../input/optiver-models-{VER}' if KAGGLE else f'./models_{VER}'
if not os.path.exists(MDLS_PATH):
    os.mkdir(MDLS_PATH)
if INFER:
    with open(f'{MDLS_PATH}/config.json', 'r') as file:
        CONFIG = json.load(file)
    CONFIG['max_trials'] = None
    CONFIG['lgb_max_trials'] = None
    CONFIG['n_jobs'] = -1 if KAGGLE else CONFIG['n_jobs']
    print('config loaded:', CONFIG)
else:
    with open(f'{MDLS_PATH}/config.json', 'w') as file:
        json.dump(CONFIG, file)
    
def seed_all(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_all(CONFIG['seed'])
start_time = time.time()

In [ ]:
target_name = 'target'
scores_folds = {}

In [ ]:
def read_train_test(path):
    train = pd.read_csv(f'{path}/train.csv')
    test = pd.read_csv(f'{path}/test.csv')
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print('train:', train.shape, '| test:', test.shape)
    return train, test

train, test = read_train_test(DATA_PATH)

In [ ]:
def calc_wap_1(df):
    wap = (
        df['bid_price1']*df['ask_size1'] + df['ask_price1']*df['bid_size1']
    ) / (
        df['bid_size1'] + df['ask_size1']
    )
    return wap

def calc_wap_2(df):
    wap = (
        df['bid_price2']*df['ask_size2'] + df['ask_price2']*df['bid_size2']
    ) / (
        df['bid_size2'] + df['ask_size2']
    )
    return wap

def log_return(series):
    return np.log(series).diff()

def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

def count_unique(series):
    return len(np.unique(series))

def book_preprocess(file_path):
    """
    Preprocess book data for each stock id
    
    """
    df = pd.read_parquet(file_path)
    df['wap1'] = calc_wap_1(df)
    df['wap2'] = calc_wap_2(df)
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    df['price_spread'] = (
        df['ask_price1'] - df['bid_price1']
    ) / (
        (df['ask_price1'] + df['bid_price1']) / 2
    )
    df['price_spread2'] = (
        df['ask_price2'] - df['bid_price2']
    ) / (
        (df['ask_price2'] + df['bid_price2']) / 2
    )
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['bid_ask_spread'] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) \
        + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs(
        (df['ask_size1'] + df['ask_size2']) \
        - (df['bid_size1'] + df['bid_size2'])
    )
    create_feature_dict = {
        'wap1': [np.sum, np.mean, np.std],
        'wap2': [np.sum, np.mean, np.std],
        'log_return1': [np.sum, realized_volatility, np.mean, np.std],
        'log_return2': [np.sum, realized_volatility, np.mean, np.std],
        'wap_balance': [np.sum, np.mean, np.std],
        'price_spread':[np.sum, np.mean, np.std],
        'price_spread2':[np.sum, np.mean, np.std],
        'bid_spread':[np.sum, np.mean, np.std],
        'ask_spread':[np.sum, np.mean, np.std],
        'total_volume':[np.sum, np.mean, np.std],
        'volume_imbalance':[np.sum, np.mean, np.std],
        'bid_ask_spread':[np.sum, np.mean, np.std],
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        """
        Function to get group stats for different windows (seconds in bucket)
        
        """
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(
            ['time_id']
        ).agg(create_feature_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(seconds_in_bucket=0, add_suffix=False)
    df_feature_400 = get_stats_window(seconds_in_bucket=400, add_suffix=True)
    df_feature_300 = get_stats_window(seconds_in_bucket=300, add_suffix=True)
    df_feature_200 = get_stats_window(seconds_in_bucket=200, add_suffix=True)
    # Merge all
    df_feature = df_feature.merge(df_feature_400, how='left', left_on='time_id_', right_on='time_id__400')
    df_feature = df_feature.merge(df_feature_300, how='left', left_on='time_id_', right_on='time_id__300')
    df_feature = df_feature.merge(df_feature_200, how='left', left_on='time_id_', right_on='time_id__200')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200'], axis=1, inplace=True)
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis=1, inplace=True)
    return df_feature

def trade_preprocess(file_path):
    """
    Function to preprocess trade data (for each stock id)
    
    """
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    create_feature_dict = {
        'log_return': [realized_volatility],
        'seconds_in_bucket': [count_unique],
        'size': [np.sum, realized_volatility, np.mean, np.std, np.max, np.min],
        'order_count': [np.mean,np.sum,np.max],
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix=False):
        """
        Function to get group stats for different windows (seconds in bucket)
        
        """
        df_feature = df[
            df['seconds_in_bucket'] >= seconds_in_bucket
        ].groupby(['time_id']).agg(create_feature_dict).reset_index()
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    df_feature = get_stats_window(seconds_in_bucket=0, add_suffix=False)
    df_feature_400 = get_stats_window(seconds_in_bucket=400, add_suffix=True)
    df_feature_300 = get_stats_window(seconds_in_bucket=300, add_suffix=True)
    df_feature_200 = get_stats_window(seconds_in_bucket=200, add_suffix=True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff / price[1:]) * 100
        power = np.sum(val * vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendency_v = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values ** 2)
        iqr_p = np.percentile(df_id['price'].values, 75) - np.percentile(df_id['price'].values, 25)
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values ** 2)
        iqr_p_v = np.percentile(df_id['size'].values, 75) - np.percentile(df_id['size'].values, 25)
        lis.append({
            'time_id': n_time_id,
            'tendency': tendency_v,
            'f_max': f_max,
            'f_min': f_min,
            'df_max': df_max,
            'df_min': df_min,
            'abs_diff': abs_diff,
            'energy': energy,
            'iqr_p': iqr_p,
            'abs_diff_v': abs_diff_v,
            'energy_v': energy_v,
            'iqr_p_v': iqr_p_v
        })
    df_lr = pd.DataFrame(lis)
    df_feature = df_feature.merge(df_lr, how='left', left_on='time_id_', right_on='time_id')
    # Merge all
    df_feature = df_feature.merge(df_feature_400, how='left', left_on='time_id_', right_on='time_id__400')
    df_feature = df_feature.merge(df_feature_300, how='left', left_on='time_id_', right_on='time_id__300')
    df_feature = df_feature.merge(df_feature_200, how='left', left_on='time_id_', right_on='time_id__200')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__400', 'time_id__300', 'time_id__200','time_id'], axis=1, inplace=True)
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis=1, inplace=True)
    return df_feature

def time_stock(df):
    """
    Function to get group stats for the stock_id and time_id
    
    """
    vol_cols = [
        'log_return1_realized_volatility', 
        'log_return2_realized_volatility', 
        'log_return1_realized_volatility_400', 
        'log_return2_realized_volatility_400', 
        'log_return1_realized_volatility_300', 
        'log_return2_realized_volatility_300', 
        'log_return1_realized_volatility_200', 
        'log_return2_realized_volatility_200', 
        'trade_log_return_realized_volatility', 
        'trade_log_return_realized_volatility_400', 
        'trade_log_return_realized_volatility_300', 
        'trade_log_return_realized_volatility_200'
    ]
    df_stock_id = df.groupby(
        ['stock_id']
    )[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')
    df_time_id = df.groupby(
        ['time_id']
    )[vol_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    df = df.merge(df_stock_id, how='left', left_on='stock_id', right_on='stock_id__stock')
    df = df.merge(df_time_id, how='left', left_on='time_id', right_on='time_id__time')
    df.drop(['stock_id__stock', 'time_id__time'], axis=1, inplace=True)
    return df
    
def preprocess(list_stock_ids, data_dir, n_jobs=8, is_train=True):
    """
    Function to make preprocessing function in parallel (for each stock id)
    
    """
    def book_n_trade(stock_id):
        if is_train:
            file_path_book = data_dir + '/book_train.parquet/stock_id=' + str(stock_id)
            file_path_trade = data_dir + '/trade_train.parquet/stock_id=' + str(stock_id)
        else:
            file_path_book = data_dir + '/book_test.parquet/stock_id=' + str(stock_id)
            file_path_trade = data_dir + '/trade_test.parquet/stock_id=' + str(stock_id)
        df_tmp = pd.merge(
            book_preprocess(file_path_book), 
            trade_preprocess(file_path_trade), 
            on='row_id', 
            how='left'
        )
        return df_tmp
    df = Parallel(n_jobs=n_jobs, verbose=1)(
        delayed(book_n_trade)(stock_id) for stock_id in list_stock_ids
    )
    df = pd.concat(df, ignore_index=True)
    return df

def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

def feval_rmspe(y_pred, lgb_train):
    """
    Function to early stop with root mean squared percentage error
    
    """
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [ ]:
train_stock_ids = train['stock_id'].unique()
train_ = preprocess(train_stock_ids, DATA_PATH, n_jobs=CONFIG['n_jobs'], is_train=True)
train = train.merge(train_, on=['row_id'], how='left')

test_stock_ids = test['stock_id'].unique()
test_ = preprocess(test_stock_ids, DATA_PATH, n_jobs=CONFIG['n_jobs'], is_train=False)
test = test.merge(test_, on=['row_id'], how='left')

train = time_stock(train)
test = time_stock(test)
print('train:', train.shape, '| test:', test.shape)
train.head()

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
# replace by order sum (tau)
train['size_tau']     = np.sqrt(1 / train['trade_seconds_in_bucket_count_unique'])
test['size_tau']      = np.sqrt(1 / test['trade_seconds_in_bucket_count_unique'])
train['size_tau_400'] = np.sqrt(1 / train['trade_seconds_in_bucket_count_unique_400'])
test['size_tau_400']  = np.sqrt(1 / test['trade_seconds_in_bucket_count_unique_400'])
train['size_tau_300'] = np.sqrt(1 / train['trade_seconds_in_bucket_count_unique_300'])
test['size_tau_300']  = np.sqrt(1 / test['trade_seconds_in_bucket_count_unique_300'])
train['size_tau_200'] = np.sqrt(1 / train['trade_seconds_in_bucket_count_unique_200'])
test['size_tau_200']  = np.sqrt(1 / test['trade_seconds_in_bucket_count_unique_200'])

In [ ]:
# tau2 
train['size_tau2']     = np.sqrt(1   / train['trade_order_count_sum'])
test['size_tau2']      = np.sqrt(1   / test['trade_order_count_sum'])
train['size_tau2_400'] = np.sqrt(.25 / train['trade_order_count_sum'])
test['size_tau2_400']  = np.sqrt(.25 / test['trade_order_count_sum'])
train['size_tau2_300'] = np.sqrt(.5  / train['trade_order_count_sum'])
test['size_tau2_300']  = np.sqrt(.5  / test['trade_order_count_sum'])
train['size_tau2_200'] = np.sqrt(.75 / train['trade_order_count_sum'])
test['size_tau2_200']  = np.sqrt(.75 / test['trade_order_count_sum'])

# delta tau
train['size_tau2_d']   = train['size_tau2_400'] - train['size_tau2']
test['size_tau2_d']    = test['size_tau2_400'] - test['size_tau2']

In [ ]:
print('train:', train.shape, '| columns:', train.columns)
print('test:', test.shape, '| columns:', test.columns)

In [ ]:
out_train = pd.read_csv(f'{DATA_PATH}/train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')
out_train = out_train.fillna(out_train.mean())
display(out_train.head())

In [ ]:
# Data separation based on knn ++
index = []
tot_dist = []
values = []
mtrx = out_train.values
scaler = MinMaxScaler(feature_range=(-1, 1))
mtrx = scaler.fit_transform(mtrx)
n_ind = int(mtrx.shape[0] / CONFIG['folds'])

# Adds index in the last column
mtrx = np.c_[mtrx, np.arange(mtrx.shape[0])]
line_number = np.random.choice(
    np.array(mtrx.shape[0]), 
    size=CONFIG['folds'], 
    replace=False
)
line_number = np.sort(line_number)[::-1]
for n in range(CONFIG['folds']):
    tot_dist.append(np.zeros(mtrx.shape[0] - CONFIG['folds']))
# Saves index
#for n in range(CONFIG['folds']):    
    values.append([line_number[n]])

s = []
for n in range(CONFIG['folds']):
    s.append(mtrx[line_number[n], :])
    mtrx = np.delete(mtrx, obj=line_number[n], axis=0)

for n in range(n_ind - 1):    
    luck = np.random.uniform(0, 1, CONFIG['folds'])    
    for cycle in range(CONFIG['folds']):
        # Saves the values of index           
        s[cycle] = np.matlib.repmat(s[cycle], mtrx.shape[0], 1)
        sum_dist = np.sum((mtrx[:, :-1] - s[cycle][:, :-1]) ** 2 , axis=1)   
        tot_dist[cycle] += sum_dist
        # Probabilities
        f = tot_dist[cycle] / np.sum(tot_dist[cycle]) # normalizing
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn += 1
        line_number[cycle] = kn
        # Delete line of the value added    
        for n_iter in range(CONFIG['folds']):
            tot_dist[n_iter] = np.delete(
                tot_dist[n_iter],
                obj=line_number[cycle], 
                axis=0
            )
            j= 0
        s[cycle] = mtrx[line_number[cycle], :]
        values[cycle].append(int(mtrx[line_number[cycle], -1]))
        mtrx = np.delete(mtrx, obj=line_number[cycle], axis=0)

for n_mod in range(CONFIG['folds']):
    values[n_mod] = out_train.index[values[n_mod]]
    
for i, x in enumerate(values):
    print(i, x[:5])

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
cols_train = list(train)
cols_train.remove('time_id')
cols_train.remove('target')
cols_train.remove('row_id')
cols_train.remove('stock_id')

In [ ]:
train.replace([np.inf, -np.inf], np.nan, inplace=True)
test.replace([np.inf, -np.inf], np.nan, inplace=True)
qt_train = []
for col in tqdm(cols_train):
    qt = QuantileTransformer(
        random_state=CONFIG['seed'],
        n_quantiles=CONFIG['n_quantiles'], 
        output_distribution='normal'
    )
    train[col] = qt.fit_transform(train[[col]])
    test[col] = qt.transform(test[[col]])    
    qt_train.append(qt)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
# Agg features
train_p = pd.read_csv(f'{DATA_PATH}/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')
corr = train_p.corr()
ids = corr.index
kmeans = KMeans(
    n_clusters=CONFIG['n_clusters'], 
    random_state=0
).fit(corr.values)
print('kmeans labels:', kmeans.labels_)
l = []
for n in range(CONFIG['n_clusters']):
    l.append ([
        (x - 1) 
        for x in ((ids + 1) * (kmeans.labels_ == n)) 
        if x > 0
    ])
mtrx = []
mtrx_test = []
n = 0
for n, ind in tqdm(enumerate(l)):
    print(n, '=>', ind)
    new_df = train.loc[train['stock_id'].isin(ind)]
    new_df = new_df.groupby(['time_id']).agg(np.nanmean)
    new_df.loc[:, 'stock_id'] = str(n) + 'c1'
    mtrx.append(new_df)
    new_df = test.loc[test['stock_id'].isin(ind)]    
    new_df = new_df.groupby(['time_id']).agg(np.nanmean)
    new_df.loc[:, 'stock_id'] = str(n) + 'c1'
    mtrx_test.append(new_df)
    n += 1
mtrx_1 = pd.concat(mtrx).reset_index()
mtrx_1.drop(columns=['target'], inplace=True)
mtrx_2 = pd.concat(mtrx_test).reset_index()

In [ ]:
del mtrx_test, mtrx, kmeans
gc.collect();

In [ ]:
mtrx_2 = pd.concat([mtrx_2, mtrx_1.loc[mtrx_1.time_id == 5]])

In [ ]:
mtrx_1 = mtrx_1.pivot(index='time_id', columns='stock_id')
mtrx_1.columns = ['_'.join(x) for x in mtrx_1.columns.ravel()]
mtrx_1.reset_index(inplace=True)
mtrx_2 = mtrx_2.pivot(index='time_id', columns='stock_id')
mtrx_2.columns = ['_'.join(x) for x in mtrx_2.columns.ravel()]
mtrx_2.reset_index(inplace=True)

In [ ]:
n_feats = [
    'time_id',
    'log_return1_realized_volatility_0c1',
    'log_return1_realized_volatility_1c1',     
    'log_return1_realized_volatility_3c1',
    'log_return1_realized_volatility_4c1',     
    'log_return1_realized_volatility_6c1',
    'total_volume_mean_0c1',
    'total_volume_mean_1c1', 
    'total_volume_mean_3c1',
    'total_volume_mean_4c1', 
    'total_volume_mean_6c1',
    'trade_size_mean_0c1',
    'trade_size_mean_1c1', 
    'trade_size_mean_3c1',
    'trade_size_mean_4c1', 
    'trade_size_mean_6c1',
    'trade_order_count_mean_0c1',
    'trade_order_count_mean_1c1',
    'trade_order_count_mean_3c1',
    'trade_order_count_mean_4c1',
    'trade_order_count_mean_6c1',      
    'price_spread_mean_0c1',
    'price_spread_mean_1c1',
    'price_spread_mean_3c1',
    'price_spread_mean_4c1',
    'price_spread_mean_6c1',   
    'bid_spread_mean_0c1',
    'bid_spread_mean_1c1',
    'bid_spread_mean_3c1',
    'bid_spread_mean_4c1',
    'bid_spread_mean_6c1',       
    'ask_spread_mean_0c1',
    'ask_spread_mean_1c1',
    'ask_spread_mean_3c1',
    'ask_spread_mean_4c1',
    'ask_spread_mean_6c1',   
    'volume_imbalance_mean_0c1',
    'volume_imbalance_mean_1c1',
    'volume_imbalance_mean_3c1',
    'volume_imbalance_mean_4c1',
    'volume_imbalance_mean_6c1',       
    'bid_ask_spread_mean_0c1',
    'bid_ask_spread_mean_1c1',
    'bid_ask_spread_mean_3c1',
    'bid_ask_spread_mean_4c1',
    'bid_ask_spread_mean_6c1',
    'size_tau2_0c1',
    'size_tau2_1c1',
    'size_tau2_3c1',
    'size_tau2_4c1',
    'size_tau2_6c1'
] 

In [ ]:
train = pd.merge(train, mtrx_1[n_feats], how='left', on='time_id')
test = pd.merge(test, mtrx_2[n_feats], how='left', on='time_id')
del mtrx_1, mtrx_2
gc.collect();

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
def objective(trial):
    params = {
        'objective': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_uniform('learning_rate', .01, .5),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', .4, 1),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', .4, 1),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 500),
        'categorical_column': [0],
        'seed': CONFIG['seed'],
        'n_jobs': CONFIG['n_jobs'],
        'verbose': 1
    }
    features = [col for col in train.columns 
                if col not in ['time_id', 'target', 'row_id']]
    y = train['target']
    oof_predictions = np.zeros(train.shape[0])
    kfold = model_selection.KFold(
        n_splits=CONFIG['folds'], 
        random_state=CONFIG['seed'], 
        shuffle=True
    )
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
        print(f'========== FOLD: {fold} ==========')
        x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train[features], y_train, weight=train_weights)
        val_dataset = lgb.Dataset(x_val[features], y_val, weight=val_weights)
        model = lgb.train(params=params,
                          num_boost_round=CONFIG['lgb_iters'],
                          train_set=train_dataset, 
                          valid_sets=[train_dataset, val_dataset], 
                          verbose_eval=int(CONFIG['lgb_iters'] / 5),
                          early_stopping_rounds=CONFIG['lgb_patience'],
                          feval = feval_rmspe)
        oof_predictions[val_ind] = model.predict(x_val[features])
    rmspe_score = rmspe(y, oof_predictions)
    print(f'OOF RMSPE: {rmspe_score}')
    return rmspe_score

def train_and_evaluate_lgb(train, test, params, infer):
    features = [col for col in train.columns 
                if col not in ['time_id', 'target', 'row_id']]
    y = train['target']
    oof_predictions = np.zeros(train.shape[0])
    test_predictions = np.zeros(test.shape[0])
    kfold = model_selection.KFold(
        n_splits=CONFIG['folds'], 
        random_state=CONFIG['seed'], 
        shuffle=True
    )
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
        print(f'========== FOLD: {fold} ==========')
        if not INFER:
            x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
            y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
            # Root mean squared percentage error weights
            train_weights = 1 / np.square(y_train)
            val_weights = 1 / np.square(y_val)
            train_dataset = lgb.Dataset(x_train[features], y_train, weight=train_weights)
            val_dataset = lgb.Dataset(x_val[features], y_val, weight=val_weights)
            model = lgb.train(params=params,
                              num_boost_round=CONFIG['lgb_iters'],
                              train_set=train_dataset, 
                              valid_sets=[train_dataset, val_dataset], 
                              verbose_eval=int(CONFIG['lgb_iters'] / 5),
                              early_stopping_rounds=CONFIG['lgb_patience'],
                              feval=feval_rmspe)
            model.save_model(f'{MDLS_PATH}/model_lgb_fold{fold}.txt', 
                             num_iteration=model.best_iteration)
            oof_predictions[val_ind] = model.predict(x_val[features])
        else:
            m_file = f'{MDLS_PATH}/model_lgb_fold{fold}.txt'
            model = lgb.Booster(model_file=m_file) 
            print('model loaded:', m_file)
        test_predictions += model.predict(test[features]) / 5
    if not infer:
        rmspe_score = rmspe(y, oof_predictions)
        print(f'OOF RMSPE: {rmspe_score}')
    else:
        rmspe_score = None
    lgb.plot_importance(model, max_num_features=10)
    return test_predictions, rmspe_score

In [ ]:
if CONFIG['lgb_max_trials']:
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=CONFIG['lgb_max_trials'])
    params = study.best_params
    print('optuna search best params:', params)
    with open(f'{MDLS_PATH}/lgb_params.json', 'w') as file:
        json.dump(params, file)
else:
    with open(f'{MDLS_PATH}/lgb_params.json', 'r') as file:
        params = json.load(file)
    print('lgb params loaded')

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
preds_lgb, rmspe_score = train_and_evaluate_lgb(train, test, params, infer=INFER)
if rmspe_score:
    with open(f'{MDLS_PATH}/results_lgb.txt', 'w') as file:
        file.write(f'OOF RMSPE: {rmspe_score}')

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
def swish(x, beta=1):
    return (x * sigmoid(beta * x))

def root_mean_squared_per_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square((y_true - y_pred)/ y_true )))

early_stopper = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=CONFIG['patience'], 
    verbose=1,
    mode='min',
    restore_best_weights=True
)
plateau_reducer = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=.1, 
    patience=CONFIG['patience'] / 2, 
    verbose=1,
    mode='min'
)
get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
hidden_units = (128, 64, 32)
stock_embedding_size = 24
cat_data = train['stock_id']

def base_model():
    stock_id_input = keras.Input(shape=(1, ), name='stock_id')
    num_input = keras.Input(shape=(362, ), name='num_data')
    stock_embedded = keras.layers.Embedding(
        max(cat_data) + 1, 
        stock_embedding_size, 
        input_length=1, 
        name='stock_embedding'
    )(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    for n_hidden in hidden_units:
        out = keras.layers.Dense(n_hidden, activation='swish')(out)
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    model = keras.Model(
        inputs = [stock_id_input, num_input],
        outputs = out
    )
    return model

def tune_model(hp, stock_id_input_dim, num_input_dim, lr=.001):
    stock_id_input = keras.Input(
        shape=(stock_id_input_dim, ), 
        name='stock_id'
    )
    num_input = keras.Input(
        shape=(num_input_dim, ), 
        name='num_data'
    )
    stock_embedded = keras.layers.Embedding(
        max(cat_data) + 1, 
        hp.Int('stock_embedding_size', 16, 64), 
        input_length=1, 
        name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    out = BatchNormalization()(out)
    out = Dropout(hp.Float('init_dropout', 0, .5))(out)
    for n_hidden in range(hp.Int('num_layers', 1, 4)):
        out = keras.layers.Dense(
            hp.Int(f'num_units_{n_hidden}', 64, 512),
            activation='swish')(out)
        out = BatchNormalization()(out)
        out = Dropout(hp.Float(f'dropout_{n_hidden}', .0, .5))(out)
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    model = keras.Model(
        inputs=[stock_id_input, num_input],
        outputs=out)
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Float('lr', .00001, .01, default=lr)
        ),
        loss=root_mean_squared_per_error)
    return model

In [ ]:
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

kf = model_selection.KFold(
    n_splits=CONFIG['folds'], 
    shuffle=True, 
    random_state=CONFIG['seed']
)
scores_folds[model_name] = []

feats = list(train)
feats.remove('time_id')
feats.remove('target')
feats.remove('row_id')
try:
    feats.remove('pred_NN')
except:
    pass
train[feats] = train[feats].fillna(train[feats].mean())
test[feats] = test[feats].fillna(train[feats].mean())
train[pred_name] = 0
test['target'] = 0

class CVTuner(kt.engine.tuner.Tuner):
    
    def run_trial(self, trial, train, feats, n_folds, 
                  batch_size=32, epochs=1, callbacks=None):
        val_losses = []
        for counter, n_count in enumerate(range(n_folds)):
            print('CV {}/{}'.format(counter + 1, n_folds))
            indexes_ = np.arange(n_folds).astype(int)    
            indexes_ = np.delete(indexes_, obj=n_count, axis=0) 
            indexes = []
            for i in range(n_folds - 1):
                indexes.extend(values[indexes_[i]])
            X_train = train.loc[train.time_id.isin(indexes), feats]
            y_train = train.loc[train.time_id.isin(indexes), target_name]
            X_test = train.loc[train.time_id.isin(values[n_count]), feats]
            y_test = train.loc[train.time_id.isin(values[n_count]), target_name]
            model = self.hypermodel.build(trial.hyperparameters)
            try:
                feats.remove('stock_id')
            except:
                pass
            num_data = X_train[feats]
            scaler = MinMaxScaler(feature_range=(-1, 1))         
            num_data = scaler.fit_transform(num_data.values)    
            cat_data = X_train['stock_id']    
            target =  y_train
            num_data_test = X_test[feats]
            num_data_test = scaler.transform(num_data_test.values)
            cat_data_test = X_test['stock_id']
            hist = model.fit(
                [cat_data, num_data], 
                target,               
                batch_size=batch_size,
                epochs=epochs,
                validation_data=([cat_data_test, num_data_test], y_test),
                callbacks=callbacks,
                validation_batch_size=len(y_test),
                shuffle=True,
                verbose=1
            )
            val_losses.append([hist.history[k][-1] for k in hist.history])
            preds = model.predict([cat_data_test, num_data_test]).reshape(1, -1)[0]
            score = round(rmspe(y_true=y_test, y_pred=preds), 5)
            print('fold {} {}: {}'.format(counter, model_name, score))
            feats.append('stock_id')
        val_losses = np.asarray(val_losses)
        self.oracle.update_trial(
            trial.trial_id, 
            {k: np.mean(val_losses[:, i]) for i, k in enumerate(hist.history.keys())}
        )
        self.save_model(trial.trial_id, model)

model_fn = lambda hp: tune_model(
        hp, 
        stock_id_input_dim=1, 
        num_input_dim=362, 
        lr=CONFIG['lr']
    )
if CONFIG['max_trials']:       
    tuner = CVTuner(
        hypermodel=model_fn,
        oracle=kt.oracles.BayesianOptimization(
            objective= kt.Objective('val_loss', direction='min'),
            num_initial_points=1,
            max_trials=CONFIG['max_trials']
        ),
        project_name=f'tuner_{VER}'
    )
    print('=' * 10, f'TUNER max trials={CONFIG["max_trials"]}', '=' * 10)
    tuner.search(
        train,
        feats,
        n_folds=CONFIG['folds'],
        batch_size=CONFIG['batch_size'],
        epochs=CONFIG['epochs'],
        callbacks=[early_stopper, plateau_reducer]
    )
    hp = tuner.get_best_hyperparameters(1)[0]
    pd.to_pickle(hp, f'{MDLS_PATH}/best_hp.pkl', protocol=4)        
else:
    hp = pd.read_pickle(f'{MDLS_PATH}/best_hp.pkl')
    print('hp params loaded')
    
elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
preds_nn = np.zeros(test.shape[0])

for counter, n_count in enumerate(range(CONFIG['folds'])):
    print('========== CV {}/{} =========='.format(counter + 1, CONFIG['folds']))
    model = model_fn(hp)
    ch_path = f'{MDLS_PATH}/model_nn_fold{counter}'
    indexes_ = np.arange(CONFIG['folds']).astype(int)    
    indexes_ = np.delete(indexes_, obj=n_count, axis=0) 
    indexes = []
    for i in range(CONFIG['folds'] - 1):
        indexes.extend(values[indexes_[i]])
    X_train = train.loc[train.time_id.isin(indexes), feats]
    y_train = train.loc[train.time_id.isin(indexes), target_name]
    X_test = train.loc[train.time_id.isin(values[n_count]), feats]
    y_test = train.loc[train.time_id.isin(values[n_count]), target_name]
    try:
        feats.remove('stock_id')
    except:
        pass
    num_data = X_train[feats]
    scaler = MinMaxScaler(feature_range=(-1, 1))         
    num_data = scaler.fit_transform(num_data.values)    
    cat_data = X_train['stock_id']    
    target =  y_train
    num_data_test = X_test[feats]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']
    if not INFER:    
        model.fit(
            [cat_data, num_data], 
            target,               
            batch_size=CONFIG['batch_size'],
            epochs=CONFIG['epochs'],
            validation_data=([cat_data_test, num_data_test], y_test),
            callbacks=[
                early_stopper, 
                plateau_reducer, 
                tf.keras.callbacks.ModelCheckpoint(
                    ch_path,
                    monitor='val_loss',
                    verbose=1, 
                    save_best_only=True,
                    save_weights_only=True, 
                    mode='min'
                )
            ],
            validation_batch_size=len(y_test),
            shuffle=True,
            verbose=1
        )
        preds = model.predict([cat_data_test, num_data_test]).reshape(1, -1)[0]
        score = round(rmspe(y_true=y_test, y_pred=preds), 5)
        print('fold {} {}: {}'.format(counter, model_name, score))
        scores_folds[model_name].append(score)
    else:
        model.load_weights(ch_path)
        print('model loaded:', ch_path)
    tt =scaler.transform(test[feats].values)
    preds_nn += model.predict([test['stock_id'], tt]).reshape(1, -1)[0].clip(0, 1e10) / CONFIG['folds']
    feats.append('stock_id')

In [ ]:
if scores_folds[model_name]:
    score = round(
        rmspe(
            y_true=train[target_name].values, 
            y_pred=train[pred_name].values
        ), 
        5
    )
    results = 'RMSPE {}: {} -> folds: {}'.format(model_name, score, scores_folds[model_name])
    print(results)
    with open(f'{MDLS_PATH}/results_nn.txt', 'w') as file:
        file.write(results)
else:
    print('*** infer mode ***')

In [ ]:
# Postprocessing
test[target_name] = .5 * preds_lgb + .5 * preds_nn
display(test[['row_id', target_name]].head())
test[['row_id', target_name]].to_csv('submission.csv', index=False)

In [ ]:
elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')